<a href="https://colab.research.google.com/github/yaswhar/learning-genesis-sim/blob/main/Muscle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install genesis-world -q
!pip install mediapy -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.7/109.7 kB 9.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 101.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.0/55.0 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/10_nvidia.json'
ICD_CONFIG_CONTENT = """{
    "file_format_version" : "1.0.0",
    "ICD" : {
        "library_path" : "libEGL_nvidia.so.0"
    }
}
"""
with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
    f.write(ICD_CONFIG_CONTENT)

In [ ]:
import genesis as gs
import numpy as np
import torch
import mediapy as media
from tqdm.notebook import tqdm

gs.init(backend=gs.cuda)

[Genesis] [22:32:20] [INFO] ╭───────────────────────────────────────────────╮
INFO:genesis:~<╭───────────────────────────────────────────────╮>~
[Genesis] [22:32:20] [INFO] │┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈ Genesis ┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈│
INFO:genesis:~<│┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈>~ ~~~~<Genesis>~~~~ ~<┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈│>~
[Genesis] [22:32:20] [INFO] ╰───────────────────────────────────────────────╯
INFO:genesis:~<╰───────────────────────────────────────────────╯>~
[Genesis] [22:32:20] [INFO] Running on [Tesla T4] with backend gs.cuda. Device memory: 14.74 GB.
INFO:genesis:Running on ~~<[Tesla T4]>~~ with backend ~~<gs.cuda>~~. Device memory: ~~<14.74>~~ GB.
[Genesis] [22:32:21] [INFO] 🚀 Genesis initialized. 🔖 version: 0.2.1, 🌱 seed: None, 📏 precision: '32', 🐛 debug: False, 🎨 theme: 'dark'.
INFO:genesis:🚀 Genesis initialized. 🔖 version: ~~<0.2.1>~~, 🌱 seed: ~~<None>~~, 📏 precision: '~~<32>~~', 🐛 debug: ~~<False>~~, 🎨 theme: '~~<dark>~~'.


In [ ]:
# Create Scene
dt = 5e-4
scene = gs.Scene(
    sim_options=gs.options.SimOptions(
        substeps=10,
        gravity=(0, 0, 0),
    ),
    viewer_options= gs.options.ViewerOptions(
        camera_pos=(1.5, 0, 0.8),
        camera_lookat=(0.0, 0.0, 0.0),
        camera_fov=40,
    ),
    mpm_options=gs.options.MPMOptions(
        dt=dt,
        lower_bound=(-1.0, -1.0, -0.2),
        upper_bound=( 1.0,  1.0,  1.0),
    ),
    fem_options=gs.options.FEMOptions(
        dt=dt,
        damping=45.,
    ),
    vis_options=gs.options.VisOptions(
        show_world_frame=False,
    ),
    show_viewer = False
)

cam = scene.add_camera(
    res    = (640, 480),
    pos    = (1.5, 0.0, 0.8),
    lookat = (0, 0, 0),
    fov    = 40,
    GUI    = False,
)

[Genesis] [22:32:55] [INFO] Scene <650aa1c> created.
INFO:genesis:Scene ~~~<<650aa1c>>~~~ created.


Most of the code is pretty standard compared to instantiating regular deformable entities. There are only two small differences that do the trick:

When instantiating soft robots `robot_mpm` and `robot_fem`, we use materials `gs.materials.MPM.Muscle` and `gs.materials.FEM.Muscle` respectively.

When stepping the simulation, we use `robot_mpm.set_actuation` or `robot_fem.set_actuation` to set the actuation of the muscle.

By default, there is only one muscle that spans the entire robot body with the muscle direction perpendicular to the ground `[0, 0, 1]`.

In [ ]:
# Add Entities
scene.add_entity(morph=gs.morphs.Plane())

E, nu = 3.e4, 0.45
rho = 1000.

robot_mpm = scene.add_entity(
    morph=gs.morphs.Sphere(
        pos=(0.5, 0.2, 0.3),
        radius=0.1,
    ),
    material=gs.materials.MPM.Muscle(
        E=E,
        nu=nu,
        rho=rho,
        model='neohooken',
    ),
)

robot_fem = scene.add_entity(
    morph=gs.morphs.Sphere(
        pos=(0.5, -0.2, 0.3),
        radius=0.1,
    ),
    material=gs.materials.FEM.Muscle(
        E=E,
        nu=nu,
        rho=rho,
        model='stable_neohooken',
    ),
)

[Genesis] [22:33:01] [INFO] Adding <gs.RigidEntity>. idx: 0, uid: <3cb5452>, morph: <gs.morphs.Plane>, material: <gs.materials.Rigid>.
INFO:genesis:Adding ~<<gs.RigidEntity>>~. idx: ~<0>~, uid: ~~~<<3cb5452>>~~~, morph: ~<<gs.morphs.Plane>>~, material: ~<<gs.materials.Rigid>>~.
[Genesis] [22:33:01] [INFO] Preprocessing geom idx 0.
INFO:genesis:Preprocessing geom idx ~~<0>~~.
[Genesis] [22:33:01] [INFO] Adding <gs.MPMEntity>. idx: 1, uid: <d2c620a>, morph: <gs.morphs.Sphere>, material: <gs.materials.MPM.Muscle>.
INFO:genesis:Adding ~<<gs.MPMEntity>>~. idx: ~<1>~, uid: ~~~<<d2c620a>>~~~, morph: ~<<gs.morphs.Sphere>>~, material: ~<<gs.materials.MPM.Muscle>>~.
[Genesis] [22:33:01] [INFO] Sampling particles with pbs-32 sampler and generating `.ptc` file:
INFO:genesis:Sampling particles with ~<pbs-32>~ sampler and generating `.ptc` file:
[Genesis] [22:33:06] [INFO] Sampled 4,159 particles.
INFO:genesis:Sampled ~~<4,159>~~ particles.
[Genesis] [22:33:06] [INFO] Adding <gs.FEMEntity>. idx: 2, 

In [ ]:
# Build
scene.build()

[Genesis] [22:33:09] [INFO] Building scene <650aa1c>...
INFO:genesis:Building scene ~~~<<650aa1c>>~~~...
[Genesis] [22:33:25] [INFO] Entity <a7bc467> added. class: FEMEntity, morph: Sphere, size: (3736, 961), material: <gs.FEM.Muscle>.
INFO:genesis:Entity <a7bc467> added. class: FEMEntity, morph: Sphere, size: (3736, 961), material: <gs.FEM.Muscle>.
[Genesis] [22:33:26] [INFO] Compiling simulation kernels...
INFO:genesis:Compiling simulation kernels...
[Genesis] [22:35:23] [INFO] Building visualizer...
INFO:genesis:Building visualizer...


In [ ]:
scene.reset()
frames = []
for i in range(1000):
    actu = np.array([0.2 * (0.5 + np.sin(0.01 * np.pi * i))])

    robot_mpm.set_actuation(actu)
    robot_fem.set_actuation(actu)
    scene.step()
    frames.append(cam.render()[0])

[Genesis] [22:35:55] [INFO] Resetting Scene <650aa1c>.
INFO:genesis:Resetting Scene ~~~<<650aa1c>>~~~.
[Genesis] [22:35:55] [INFO] Running at 61.93 FPS.
INFO:genesis:Running at ~<61.93>~ FPS.
[Genesis] [22:35:56] [INFO] Running at 24.57 FPS.
INFO:genesis:Running at ~<24.57>~ FPS.
[Genesis] [22:35:56] [INFO] Running at 21.95 FPS.
INFO:genesis:Running at ~<21.95>~ FPS.
[Genesis] [22:35:56] [INFO] Running at 20.59 FPS.
INFO:genesis:Running at ~<20.59>~ FPS.
[Genesis] [22:35:56] [INFO] Running at 18.96 FPS.
INFO:genesis:Running at ~<18.96>~ FPS.
[Genesis] [22:35:56] [INFO] Running at 18.00 FPS.
INFO:genesis:Running at ~<18.00>~ FPS.
[Genesis] [22:35:56] [INFO] Running at 17.16 FPS.
INFO:genesis:Running at ~<17.16>~ FPS.
[Genesis] [22:35:57] [INFO] Running at 16.71 FPS.
INFO:genesis:Running at ~<16.71>~ FPS.
[Genesis] [22:35:57] [INFO] Running at 16.64 FPS.
INFO:genesis:Running at ~<16.64>~ FPS.
[Genesis] [22:35:57] [INFO] Running at 16.71 FPS.
INFO:genesis:Running at ~<16.71>~ FPS.
[Genesi

In [ ]:
media.show_video(frames, fps=200)

This browser does not support the video tag.
